In [ ]:
import os
import json
import torch
from torch.utils.tensorboard import SummaryWriter
from utils import rinomina_classi_in_dir, undersample_dataset, count_images_per_class, split_train_val_test
from data_loader import get_data_loaders
from model import CustomCNN, TransferMobileNet
from train import train_model
from evaluate import evaluate_model, plot_training
from jsonschema import validate
import jsonschema

In [ ]:
# Caricamento del file di configurazione
with open('config.json', 'r') as f:
    config = json.load(f)

In [ ]:
# Caricamento del JSON Schema
with open('config_schema.json', 'r') as f:
    schema = json.load(f)

# Validazione della configurazione
try:
    validate(instance=config, schema=schema)
    print("Configurazione valida.")
except jsonschema.exceptions.ValidationError as err:
    print("Errore nella configurazione:", err)
    raise

In [ ]:
# Configurazione
original_dir = config.get("original_data_path", "./Dataset/PlantVillage")
balanced_dir = config.get("balanced_data_path", "./Dataset/PlantVillage_balanced")
split_dir = config.get("split_data_path", "./Dataset/PlantVillage_split")

train_dir = config["train_data_path"]  
val_dir = config["val_data_path"]      
test_dir = config["test_data_path"]    

batch_size = config.get("batch_size", 64)
val_ratio = 0.15     
test_ratio = 0.15      
epochs = config.get("num_epochs", 10)
learning_rate = config.get("learning_rate", 0.001)
target_accuracy = config.get("target_accuracy", 0.95)
early_stopping_patience = config.get("early_stopping_patience", 5)

print(f"Original data dir: {original_dir}")
print(f"Balanced data dir: {balanced_dir}")
print(f"Split data dir: {split_dir}")
print(f"Training dir (loader): {train_dir}")
print(f"Validation dir (loader): {val_dir}")



In [ ]:
# Rinomina classi
rinomina_classi_in_dir(original_dir)

In [22]:
# Undersampling per bilanciare
undersample_dataset(original_dir, balanced_dir, max_per_class=1000)

✅ Dataset ridotto salvato in: ./Dataset/PlantVillage_balanced


In [23]:
# Conta immagini
print("Distribuzione originale:", count_images_per_class(original_dir))
print("Distribuzione bilanciata:", count_images_per_class(balanced_dir))

Distribuzione originale: {'pep_bell_Bacterial_spot': 976, 'pep_bell_healthy': 1478, 'pot_Early_blight': 1000, 'pot_healthy': 152, 'pot_Late_blight': 1000, 'tom_Bacterial_spot': 2127, 'tom_Early_blight': 1000, 'tom_healthy': 1591, 'tom_Late_blight': 1909, 'tom_Leaf_Mold': 952, 'tom_Septoria_leaf_spot': 1771, 'tom_Spider_mites_Two_spotted_spider_mite': 1676, 'tom_Target_Spot': 1404, 'tom_Tomato_mosaic_virus': 373, 'tom_Tomato_YellowLeaf_Curl_Virus': 3208}
Distribuzione bilanciata: {'pep_bell_Bacterial_spot': 976, 'pep_bell_healthy': 1000, 'pot_Early_blight': 1000, 'pot_healthy': 152, 'pot_Late_blight': 1000, 'tom_Bacterial_spot': 1000, 'tom_Early_blight': 1000, 'tom_healthy': 1000, 'tom_Late_blight': 1000, 'tom_Leaf_Mold': 952, 'tom_Septoria_leaf_spot': 1000, 'tom_Spider_mites_Two_spotted_spider_mite': 1000, 'tom_Target_Spot': 1000, 'tom_Tomato_mosaic_virus': 373, 'tom_Tomato_YellowLeaf_Curl_Virus': 1000}


In [24]:
# Split train/val/test
split_train_val_test(balanced_dir, split_dir, val_ratio=val_ratio, test_ratio=test_ratio)
print("Dataset splittato in train/val/test.")

Dataset splittato in train/val/test.


In [25]:
# DataLoader
train_loader, val_loader, class_names = get_data_loaders(split_dir, img_size=(224, 224), batch_size=batch_size)
print(f"Classi: {class_names}")


Classi: ['pep_bell_Bacterial_spot', 'pep_bell_healthy', 'pot_Early_blight', 'pot_Late_blight', 'pot_healthy', 'tom_Bacterial_spot', 'tom_Early_blight', 'tom_Late_blight', 'tom_Leaf_Mold', 'tom_Septoria_leaf_spot', 'tom_Spider_mites_Two_spotted_spider_mite', 'tom_Target_Spot', 'tom_Tomato_YellowLeaf_Curl_Virus', 'tom_Tomato_mosaic_virus', 'tom_healthy']


In [26]:
# Creazione modello e device

model = TransferMobileNet(num_classes=len(class_names))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)

Using device: cuda


In [27]:
# Carico il CheckPoint
checkpoint_path = os.path.join(config.get("checkpoint_dir", "./checkpoints"), "model_best.pth")

In [ ]:
# Chiamata Training
model, train_losses, val_losses, train_accs, val_accs = train_model(
    model,
    train_loader,
    val_loader,
    device,
    epochs,
    learning_rate,
    early_stopping_patience=early_stopping_patience,
    target_accuracy=target_accuracy,
    checkpoint_dir=config.get("checkpoint_dir", "./checkpoints"),
    resume_from=None,  
    log_dir=config.get("log_dir", "./runs")
)


#tensorboard --logdir=./runs

Epoch 1/10 - Train loss: 1.4750, Train acc: 0.5665 - Val loss: 0.7454, Val acc: 0.7926
Epoch 2/10 - Train loss: 0.9024, Train acc: 0.7293 - Val loss: 0.5527, Val acc: 0.8367
Epoch 3/10 - Train loss: 0.7865, Train acc: 0.7533 - Val loss: 0.4811, Val acc: 0.8571
Epoch 4/10 - Train loss: 0.7372, Train acc: 0.7659 - Val loss: 0.4136, Val acc: 0.8705
Epoch 5/10 - Train loss: 0.7191, Train acc: 0.7692 - Val loss: 0.3817, Val acc: 0.8849
Epoch 6/10 - Train loss: 0.6764, Train acc: 0.7817 - Val loss: 0.3908, Val acc: 0.8685
Epoch 7/10 - Train loss: 0.6838, Train acc: 0.7834 - Val loss: 0.3848, Val acc: 0.8700
Epoch 8/10 - Train loss: 0.6611, Train acc: 0.7832 - Val loss: 0.3561, Val acc: 0.8819
Epoch 9/10 - Train loss: 0.6583, Train acc: 0.7843 - Val loss: 0.3454, Val acc: 0.8933


In [ ]:
# Visualizza andamento training
plot_training(train_losses, val_losses, train_accs, val_accs)


In [ ]:
# Valuta il modello su validation set
evaluate_model(model, val_loader, device, class_names)

In [ ]:
# Salvataggio Modello finale
torch.save(model.state_dict(), "plantvillage_transfer_model.pth")
print("✅ Modello salvato in plantvillage_transfer_model.pth")